In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
names = [i.split(".")[0] for i in os.listdir("data")]
names

['ahmedabad',
 'bangalore',
 'chennai',
 'gurugram',
 'hyderabad',
 'indore',
 'jaipur',
 'mumbai',
 'noida',
 'pune']

In [3]:
data = pd.concat([pd.read_csv(f"data/{i}.csv", index_col=0) for i in names], ignore_index=True)
data


,company_name,company_rating,other_data
0,Zydus Lifesciences,4.2,"Pharma , 10k-50k Employees , Public , 72 years..."
1,Intas Pharmaceuticals,4.2,"Pharma , 10k-50k Employees , Fortune India 500..."
2,Vodafone Idea,4.2,"Telecom , 5k-10k Employees , Public , 6 years ..."
3,TTEC India,3.8,"BPO , 1k-5k Employees , Public , 40 years old ..."
4,Cadila Pharmaceuticals,3.6,"Pharma , 5k-10k Employees , Public , 73 years ..."
...,...,...,...
94575,Preethi Kitchen Appliances,4.2,"Electronics Manufacturing , 1k-5k Employees , ..."
94576,ACCURUB Technologies,4.0,"Auto Components , 51-200 Employees , 13 years ..."
94577,HLL Lifecare,3.7,"Healthcare , 1k-5k Employees , Central , 58 ye..."
94578,Singularity AIX,3.7,Pune +2 more


In [4]:
other_data = data["other_data"].apply(lambda x: [i.strip() for i in x.split(",")])
other_data

0        [Pharma, 10k-50k Employees, Public, 72 years o...
1        [Pharma, 10k-50k Employees, Fortune India 500,...
2        [Telecom, 5k-10k Employees, Public, 6 years ol...
3        [BPO, 1k-5k Employees, Public, 40 years old, A...
4        [Pharma, 5k-10k Employees, Public, 73 years ol...
                               ...                        
94575    [Electronics Manufacturing, 1k-5k Employees, 4...
94576    [Auto Components, 51-200 Employees, 13 years o...
94577    [Healthcare, 1k-5k Employees, Central, 58 year...
94578                                       [Pune +2 more]
94579                                       [Pune +3 more]
Name: other_data, Length: 94580, dtype: object

In [20]:
def clean_other_data(data):
    li = data.copy()

    # location
    location = li.pop().split("+")[0].strip().title()

    # years_old
    hasYears = ["years" in i.lower() for i in li]
    if True in hasYears:
        years = int(li.pop(hasYears.index(True)).split(" ")[0])
    else:
        years = np.nan

    # employees
    hasEmployees = ["employees" in i.lower() for i in li]
    if True in hasEmployees:
        employees = li.pop(hasEmployees.index(True)).replace(" Lakh", "00000").replace("k", "000").split(" ")[0]
    else:
        employees = np.nan

    # industry and type
    if len(li) == 2:
        industry = li[0]
        company_type = li[1]
    elif len(li) == 1:
        industry = li[0]
        company_type = np.nan
    else:
        industry = np.nan
        company_type = np.nan

    return {"industry": industry, "size": employees, "type": company_type, "years_old": years, "location": location}

In [21]:
cleaned_other_data = pd.DataFrame([clean_other_data(i) for i in other_data])
cleaned_other_data

,industry,size,type,years_old,location
0,Pharma,10000-50000,Public,72.0,Ahmedabad
1,Pharma,10000-50000,Fortune India 500,47.0,Ahmedabad
2,Telecom,5000-10000,Public,6.0,Ahmedabad
3,BPO,1000-5000,Public,40.0,Ahmedabad
4,Pharma,5000-10000,Public,73.0,Ahmedabad
...,...,...,...,...,...
94575,Electronics Manufacturing,1000-5000,NaN,46.0,Pune
94576,Auto Components,51-200,NaN,13.0,Pune
94577,Healthcare,1000-5000,Central,58.0,Pune
94578,NaN,NaN,NaN,NaN,Pune


In [19]:
'abcd'.replace("a", "b")

'bbcd'

In [24]:
cleaned_other_data["size"].value_counts()

size
51-200          14550
100000+         12974
1000-5000        9141
11-50            7686
201-500          7335
501-1000         5346
50000-100000     3582
10000-50000      3231
5000-10000       2177
1-10             1091
Name: count, dtype: int64

In [25]:
cleaned_data = data[["company_name", "company_rating"]].join(cleaned_other_data)
cleaned_data

,company_name,company_rating,industry,size,type,years_old,location
0,Zydus Lifesciences,4.2,Pharma,10000-50000,Public,72.0,Ahmedabad
1,Intas Pharmaceuticals,4.2,Pharma,10000-50000,Fortune India 500,47.0,Ahmedabad
2,Vodafone Idea,4.2,Telecom,5000-10000,Public,6.0,Ahmedabad
3,TTEC India,3.8,BPO,1000-5000,Public,40.0,Ahmedabad
4,Cadila Pharmaceuticals,3.6,Pharma,5000-10000,Public,73.0,Ahmedabad
...,...,...,...,...,...,...,...
94575,Preethi Kitchen Appliances,4.2,Electronics Manufacturing,1000-5000,NaN,46.0,Pune
94576,ACCURUB Technologies,4.0,Auto Components,51-200,NaN,13.0,Pune
94577,HLL Lifecare,3.7,Healthcare,1000-5000,Central,58.0,Pune
94578,Singularity AIX,3.7,NaN,NaN,NaN,NaN,Pune


In [46]:
cleaned_data.drop_duplicates(inplace=True, ignore_index=True)

In [47]:
cleaned_data.to_csv("cleaned_data.csv")